In [48]:
from langchain.document_loaders import DirectoryLoader, UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

In [49]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [50]:
loader = DirectoryLoader("2023-07-26/en-US/")

In [51]:
documents = loader.load()

In [71]:
len(documents)

327

In [53]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

In [56]:
len(texts)

2043

In [57]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

persist_directory = "./storage"
# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embeddings,
                                 persist_directory=persist_directory)
vectordb.persist()

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

retriever = vectordb.as_retriever(search_kwargs={"k": 5})
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [69]:
user_input = 'How to setup 2fa in titan mail app'

query = f"###Prompt {user_input}"

llm_response = qa(query)
print(llm_response["result"])

To set up Two-Factor Authentication (2FA) in the Titan Mail app, please follow these steps:

1. Ensure that you have the latest version of the Titan Mail app installed on your Android or iOS device.

2. Open the Titan Mail app and log in to your account using your username and password.

3. Once you are logged in, navigate to the settings or account preferences section of the app.

4. Look for the option to enable Two-Factor Authentication or 2FA. This option may be under the security or privacy settings.

5. Click on the enable or setup button for Two-Factor Authentication.

6. Follow the on-screen instructions to complete the setup process. This may involve verifying your phone number, linking a trusted device, or setting up a backup method.

7. Once you have successfully set up Two-Factor Authentication, the app will prompt you to verify your identity using a second factor, such as a code sent to your phone or a biometric scan, each time you log in.

Remember to keep your backup cod

In [73]:
new_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
